In [5]:
import pandas as pd
import os

In [6]:
print('current path',os.path.abspath('.'))

current path F:\陶士来文件\tsl_python_project\titanic_disaster


In [37]:
data=pd.read_csv(r'F:\陶士来文件\tsl_python_project\python_base\pandas\train.csv',
               usecols=[ 'Name','Sex','Age', 'PassengerId',  'Survived',  'Pclass'])
print(data.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  
0                            Braund, Mr. Owen Harris    male  22.0  
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0  
2                             Heikkinen, Miss. Laina  female  26.0  
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0  
4                           Allen, Mr. William Henry    male  35.0  


In [33]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age'], dtype='object')

In [30]:
###groupby的几种使用形式，以后补充

get_count = lambda x:x.count()##原例以后补充
data_new=data['PassengerId'].groupby(data['Age']).agg({'maxValue':'max','minValue':'min','filterCount':get_count}).reset_index()
#add_suffix(suffix)，add_prefix(prefix)添加前缀和后缀
print(data_new.head())

    Age  maxValue  minValue  filterCount
0  0.42       804       804            1
1  0.67       756       756            1
2  0.75       645       470            2
3  0.83       832        79            2
4  0.92       306       306            1


F:\陶士来文件\software\python3_6_5\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [34]:
# 为什么要用agg：
# 如果你不用agg，也可以实现同样的功能，因为Pandas自带了max、min、mean这些聚合函数，用法如下

In [38]:
col1='PassengerId'
col2='Age'

data00 = data[col1].groupby(data[col2]).max()
data11 = data[col1].groupby(data[col2]).mean()
data22 = data[col1].groupby(data[col2]).min()
# 但是这样既笨重，又没办法通过自定义的聚合函数来实现自定义统计，
# 而用agg可以传入自定义的聚合函数

In [39]:
print(data00.head(),data11.head(),data22.head())


Age
0.42    804
0.67    756
0.75    645
0.83    832
0.92    306
Name: PassengerId, dtype: int64 Age
0.42    804.0
0.67    756.0
0.75    557.5
0.83    455.5
0.92    306.0
Name: PassengerId, dtype: float64 Age
0.42    804
0.67    756
0.75    470
0.83     79
0.92    306
Name: PassengerId, dtype: int64


In [ ]:
# 为什么agg里面要放字典
# agg里面可以不传字典，但是新生成的列是不能直接生成你想要的列名的。如果你想要定义列名，那你得这么做：
#先初始化列为1，再给这一列传入统计后的值

data['max'] = 1
data['max'] = data[col1].groupby(data[col2]).agg('max').reset_index()
print(data)
add_prefix()
# 用这种方法，如果同时实现最大值、最小值、均值、计数等参数统计的话，需要写八行代码，
# 而在agg中传入字典参数，同时定义列名和所用的聚合函数，只需要一列就能完成了，甚好。

In [42]:
# 为什么要写reset_index
# 这是因为 比如groupby 用户字段，那么 用户字段会默认认定为是index,这并不符合预期，因为往往用户字段应该还原为普通列字段，方便后面和其他含用户字段的表的join或merge
# 
# 如果是分组字段是多个，被分组统计字段是一个(或多个)时，用上面的模板好像有点问题：
# 要用到unstack()

import pandas as pd
import numpy as np
df=pd.DataFrame({'uid':[1001,1001,1002,1003,1003,1003],'tran_type':[0,1,0,1,0,1],'amount':[1,2,3,4,np.NaN,5]})


print(df.groupby(['uid','tran_type'])['amount'].mean())


print(df.groupby(['uid','tran_type'])['amount'].mean().unstack())
	
df=df.groupby(['uid','tran_type'])['amount'].mean().unstack()
print(df)
df.columns=['tran_type_0','tran_type_1']
df.reset_index()
print(df)




uid   tran_type
1001  0            1.0
      1            2.0
1002  0            3.0
1003  0            NaN
      1            4.5
Name: amount, dtype: float64
tran_type    0    1
uid                
1001       1.0  2.0
1002       3.0  NaN
1003       NaN  4.5
tran_type    0    1
uid                
1001       1.0  2.0
1002       3.0  NaN
1003       NaN  4.5
      tran_type_0  tran_type_1
uid                           
1001          1.0          2.0
1002          3.0          NaN
1003          NaN          4.5


In [43]:
# 这样的数据差不多是我们想要的了
# 但是总感觉有点麻烦，
# 能不能用agg的方法，试了一下
df=pd.DataFrame({'uid':[1001,1001,1002,1003,1003,1003],'tran_type':[0,1,0,1,0,1],'amount':[1,2,3,4,np.NaN,5]})

In [45]:
print(df.groupby(['uid','tran_type'])['amount'].agg({'mean'}).unstack())

          mean     
tran_type    0    1
uid                
1001       1.0  2.0
1002       3.0  NaN
1003       NaN  4.5


In [48]:
# 			mean
# tran_type	0     	1
# uid		
# 1001	   1.0      2.0
# 1002	   3.0     NaN
# 1003	  NaN	4.5
# 牵扯到了多级索引了
# 
# 如何转化为大致 
#    	uid	tran_type_0	tran_type_1
#    0	1001	1.0	        2.0
#    1	1002	3.0	        NaN
#    2	1003	NaN	4.5
#    的样子呢
# --------------------- 
# 作者：波波虾遁地兽 
# 来源：CSDN 
# 原文：https://blog.csdn.net/zlb872551601/article/details/84800334 
# 版权声明：本文为博主原创文章，转载请附上博文链接！

In [51]:
df=df[['uid','amount']].groupby(['uid']).agg({'mean','min'})
# 		amount
# 	mean	min
# uid		
# 1001	1.5	1.0
# 1002	3.0	3.0
# 1003	4.5	4.0
# 可读性还好，但是这没办法和后面的含有uid的表join或merge吧
# 因为是多级索引，要是能降级就好了，
df.columns
# MultiIndex(levels=[['amount'], ['mean', 'min']],
#            labels=[[0, 0], [0, 1]])
# 一种想法是直接去掉一级就好了
#直接去掉一层索引，df.columns = df.columns.droplevel(0)，这是鸵鸟策略吧
#df.columns=['uid','amount_mean','amount_min']
#第二种想法手动去写列名，是不是显得不够专业啊，下面大神给出的# 把2层合并到一层的 骚写法，
df.columns = ["_".join(x) for x in df.columns.ravel()]
print(df.reset_index())
# uid	amount_mean	amount_min
# 0	1001	1.5	1.0
# 1	1002	3.0	3.0
# 2	1003	4.5	4.0
# 这样就是我们想要的数据了


    uid  amount_mean  amount_min
0  1001          1.5         1.0
1  1002          3.0         3.0
2  1003          4.5         4.0


In [54]:
# 再次回到我们开始提出的问题，能不能用agg的方法对groupby多个字段，
# 同样用到上面的对多级索引降级的方式就可以很好解决 问题了
df=pd.DataFrame({'uid':[1001,1001,1002,1003,1003,1003],'tran_type':[0,1,0,1,0,1],'amount':[1,2,3,4,np.NaN,5]})
df=df.groupby(['uid','tran_type'])['amount'].agg({'mean'}).add_prefix('amount_').unstack()
# 		amount_mean
# tran_type	0	1
# uid		
# 1001	 1.0	     2.0
# 1002	 3.0	     NaN
# 1003	 NaN     4.5
# 使用层次索引合并降级
df.columns = [x[0]+"_"+str(x[1]) for x in df.columns.ravel()]
#这里因为0是数值型，直接和字符串amount_mean不能用"_".join(x) 方式连接
df.columns
print(df.reset_index())
# 	uid	amount_mean_0	amount_mean_1
# 0	1001	1.0	                 2.0
# 1	1002	3.0	                NaN
# 2	1003	NaN	          4.5


    uid  amount_mean_0  amount_mean_1
0  1001            1.0            2.0
1  1002            3.0            NaN
2  1003            NaN            4.5


In [56]:
# 这正是我们想要的
# 
# 和pandas.groupby常一起用的还有transform函数和describe函数，cut函数，
# 以及计数的size()函数(计np.NaN)和count()函数（不计np.NaN）
# 
# describe函数比较简单易懂，常见统计量的summary
# transform函数则在于，在原df的索引index不变下方便的在原df中增加groupby之后的聚合结果
# 
# 前面进行聚合运算的时候，得到的结果是一个以分组名为 index 的结果对象。如果我们想使用原数组的 index 的话，
# 就需要进行 merge 转换。transform(func, args, *kwargs) 方法简化了这个过程，
# 它会把 func 参数应用到所有分组，然后把结果放置到原数组的 index 上（如果结果是一个标量，就进行广播）：
# 即transform隐式地帮我们完成了一次merge操作，
df['count_B']=df.groupby(['group1', 'group2'])['B'].transform('count')
#1
# 上面运算的结果分析： {‘group1’:’A’, ‘group2’:’C’}的组合共出现3次，即index为0,1,2。对应”B”列的值分别是
# ”one”,”NaN”,”NaN”，由于count()计数时不包括Nan值，因此{‘group1’:’A’, ‘group2’:’C’}的count计数值为1。
# transform()方法会将该计数值在dataframe中所有涉及的rows都显示出来（我理解应该就进行广播）
# 
# 对连续数值变化的列做分组，会用到cut函数，cut函数中需自定义bins的范围
# 
# 聚合方法size()和count()
# size跟count的区别： size计数时包含NaN值，而count不包含NaN值

In [57]:
#unstack用法
# groupby（）可以根据DataFrame中的某一列或者多列内容进行分组聚合，
# 当DataFrame聚合后为两列索引时，可以使用unstack()将聚合的两列中一列值调整为行索引，另一列的值调整为列索引

In [58]:
test_df = pd.DataFrame({ 'col_1':['a', 'a', 'b', 'a', 'a', 'b', 'c', 'a', 'c'],
                                      'col_2':['d', 'd', 'd', 'e', 'f', 'e', 'd', 'f', 'f'],
                                      'col_3':[ 1,  2,  3,   1,  4,  5,  6,  4,  5]})


In [63]:
df1=test_df.groupby(['col_1', 'col_2']).count()
print(df1)
print(df1.index)
print(df1.columns)

             col_3
col_1 col_2       
a     d          2
      e          1
      f          2
b     d          1
      e          1
c     d          1
      f          1
MultiIndex(levels=[['a', 'b', 'c'], ['d', 'e', 'f']],
           codes=[[0, 0, 0, 1, 1, 2, 2], [0, 1, 2, 0, 1, 0, 2]],
           names=['col_1', 'col_2'])
Index(['col_3'], dtype='object')


In [64]:
# 对分组聚合后的数据进行unstack
df1=test_df.groupby(['col_1', 'col_2']).count().unstack()
print(df1)
print(df1.index)
print(df1.columns)

      col_3          
col_2     d    e    f
col_1                
a       2.0  1.0  2.0
b       1.0  1.0  NaN
c       1.0  NaN  1.0
Index(['a', 'b', 'c'], dtype='object', name='col_1')
MultiIndex(levels=[['col_3'], ['d', 'e', 'f']],
           codes=[[0, 0, 0], [0, 1, 2]],
           names=[None, 'col_2'])


In [65]:
# 对分组聚合后的某列进行unstack
df1=test_df.groupby(['col_1', 'col_2']).count()['col_3'].unstack()
print(df1)
print(df1.index)
print(df1.columns)

col_2    d    e    f
col_1               
a      2.0  1.0  2.0
b      1.0  1.0  NaN
c      1.0  NaN  1.0
Index(['a', 'b', 'c'], dtype='object', name='col_1')
Index(['d', 'e', 'f'], dtype='object', name='col_2')
